# Analysis of Forcing Fields

`mom6_tools.MOM6grid` returns an object with MOM6 grid data.

`mom6_tools.latlon_analysis` has a collection of tools used to perform spatial analysis  (e.g., time averages and spatial mean).

The goal of this notebook is the following:

1) serve as an example of how to post-process CESM/MOM6 output;

2) create time averages of forcing fields;

3) create time-series of globally-averaged forcing fields;

4) compare model results vs. observations (TODO);

5) compare model results vs. another model results (TODO).

In [1]:
%matplotlib inline
from mom6_tools.MOM6grid import MOM6grid
from mom6_tools.DiagsCase import DiagsCase
from mom6_tools.m6toolbox import genBasinMasks
from mom6_tools.latlon_analysis import time_mean_latlon
import matplotlib.pyplot as plt
import numpy as np
import yaml

Basemap module not found. Some regional plots may not function properly


In [2]:
# Read in the yaml file
diag_config_yml_path = "/glade/u/home/gmarques/libs/mom6-tools/docs/source/examples/diag_config.yml"
diag_config_yml = yaml.load(open(diag_config_yml_path,'r'), Loader=yaml.Loader)

In [3]:
# Create the case instance
dcase = DiagsCase(diag_config_yml['Case'])
RUNDIR = dcase.get_value('RUNDIR')
print('Run directory is:', RUNDIR)
print('Casename is:', dcase.casename)

Run directory is: /glade/scratch/gmarques/g.c2b6.GJRA.TL319_t061.long_JRA_mct.076/run
Casename is: g.c2b6.GJRA.TL319_t061.long_JRA_mct.076


In [4]:
# create an empty class object
class args:
  pass

args.case_name = dcase.casename
args.start_date = ''
args.end_date = ''
args.savefigs = False

In [5]:
# set avg dates
avg = diag_config_yml['Avg']
if not args.start_date : args.start_date = avg['start_date']
if not args.end_date : args.end_date = avg['end_date']

In [9]:
# The following parameters must be set accordingly
######################################################
# name of forcing file
forcing = dcase.casename+'.mom6.hm*.nc'
######################################################


args.infile = RUNDIR+'/'+forcing
# array with 2D forcing variables to be processed
args.variables = ['hfds','PRCmE','taux','tauy']
args.savefigs = False
args.time_series = True
args.nw = 6 # number of workers

In [10]:
# Load the grid
grd = dcase.grid
depth = grd.depth_ocean
# remove Nan's, otherwise genBasinMasks won't work
depth[np.isnan(depth)] = 0.0
basin_code = genBasinMasks(grd.geolon, grd.geolat, depth, verbose=False)

11.16427964664149 64.78854837178643 [391, 434]


In [11]:
%matplotlib inline
# plot time averages. If variables is NOT specified, all 2D variables will be plotted.
#time_mean_latlon_plot(args,grd, variables=args.variables)
time_mean_latlon(args,grd,args.variables)

Requesting 6 workers... 

/proxy/45255/status


ValueError: cannot infer dimension to concatenate: supply the ``concat_dim`` argument explicitly